This is a somewhat straightforward, perhaps even naive calculation of determinants via the recursive approach mentioned here:   
http://linear.ups.edu/fcla/section-DM.html

The point of this code is for your author (or outside user) to better 'visualize' the definition of a determinant.  This code is a way to put that into practice.  This code, of course, should not be used for n > 15 as it grows O(n!)  

One particularly nice thing about this, is if we interpret our $n$ x $n$ matrices as graphs, we can see that this is calculating paths that visit one (weighted) destination, and it chains the weights together and in fact each full 'pass' terminate at the point of creating a full cycle -- the approach is infact awfully close to depth first search.  After going through and 'trying' each and every full cycle, our formula sums all these chains.  The one 'peculiar' thing is line 52 in the code which has 

    ones_alternating *= -1

This is what makes the determinant tractable and in line with geometric intuition / inspiration -- and of course can be used in linear independence tests, developing the characteristic polynomial and so on.  On the other hand, this alternating minus sign is also, strictly speaking, where it departs from a permanent. 



In [1]:
import numpy as np
import numba
np.set_printoptions(precision = 2, linewidth=180)

In [2]:
@numba.jit(nopython = True)
def create_sub_matrix(A, m, n, k):
    ### the below routine is used to in effect slice off row zero, 
    # and then slice off different sub matrices
    # i did not see a particularly great way of doing this directly with numpy indexing
    row_index = -1
    sliced_A = np.zeros((m - 1, n - 1))

    for i in range(1,m):
        # recall that we always skip row zero
        row_index += 1                
        col_index = -1
        
        for j in range(n):                    
            if j == k:
                continue
            col_index += 1
            sliced_A[row_index, col_index] += A[i,j]
    return sliced_A


def compute_determinant_recursively(A, level= 0):
    """
    This is the main function for computing the determinant of a square matrix
    in a 'textbook' manner
    """
    m, n = A.shape
    #######
    # it's a common convention to not go all the way down to a 1 x 1 matrix, 
    # and instead have the 2 x 2 as your base case
    # however, this whole process is here for thought experiment purposes, 
    # so I tweaked it to go the extra step to the 1 x 1
    
    # conventional base case, is below
    #     if m == 2 and n == 2:
    #    return A[0,0] * A[1,1] - A[1,0] * A[0,1]
    #######
    
    if m == 1 and n == 1:
        # base case
        return A[0,0]
    else:
        # recursive casse
        running_sum = 0
        ones_alternating = -1
        
        for k in range(A.shape[1]):            
            sub_matrix = create_sub_matrix(A, m, n, k)
#             print("level = ", level)
#             print(sub_matrix, "\n",)
            
            ones_alternating *= -1
            scalar = ones_alternating * A[0,k]            
            increment = compute_determinant_recursively(sub_matrix, level + 1)
            running_sum += scalar * increment
        return running_sum


In [3]:
n = 4

# A = np.random.randn((n**2))
A = np.random.rand(n**2)
A
A = np.reshape(A,(n,n))
A

# A = np.array([[ 3,  2, -1],
#               [ 4,  1,  6],
#               [-3, -1,  2]])

array([[ 0.18,  0.8 ,  0.54,  0.12],
       [ 0.18,  0.88,  0.77,  0.18],
       [ 0.5 ,  0.92,  0.06,  0.57],
       [ 0.82,  0.75,  0.92,  0.29]])

In [4]:
np.linalg.det(A)

-0.053044159878338576

In [5]:
m = 2
A = np.random.randint(0, 2, m**2)
A = A.reshape(m,m)
A

array([[0, 0],
       [1, 0]])

In [6]:
int(round(2.1%2,0))

0

The below has some interesting things about determinants over reals vs gf(2) of random matrices that only have 0 or 1 in them.  


In [7]:
K_trials = 50000
m = 25
gf2_hits = 0
reals_hits = 0

for _ in range(K_trials):    
    A = np.random.randint(0, 2, m**2)
    A = A.reshape(m,m)
    result = int(round(np.linalg.det(A), 0))
    if result % 2 !=0:        
        # i.e. a 'hit' is when we have a nonsingular matrix
        gf2_hits += 1
    if result != 0:
        reals_hits += 1
        
print("the nonsingular percentage over gf2 is: ")
print(gf2_hits / K_trials, "\n")
print('the nonsingular percentage over reals is: ')
print(reals_hits/ K_trials)

    
    

the nonsingular percentage over gf2 is: 
0.28868 

the nonsingular percentage over reals is: 
0.99996


The below is a mostly unrelated routine, geared toward calculating determinants of symbolic functions that return values over Reals (or perhaps complex numbers)


In [8]:
import sympy as sp
from sympy import sqrt
from sympy import sin
from fractions import Fraction

In [9]:
x = sp.Symbol('x')
y = sp.Symbol('y')
z = sp.Symbol('z')

In [10]:
def get_char_pol_sym(A):
    """
    This uses Newton's identities to get the characteristic polynomial,
    (valid over fields with characteristic zero)
    adjusted for sympy
    """
    assert(A.shape[0] == A.shape[1]) # it must be a square matrix
    n = A.shape[0]
    
    # slightly different syntax for making a vector of zeros in sympy
    charpoly_array = sp.zeros(n + 1, 1)
    traces_array = sp.zeros(n + 1, 1)
    
    traces_array[0] = n
    working_matrix = A.copy()
    
    traces_array[1] = sp.trace(working_matrix)
    tracer_idx = 1
    # note trace (A^1) will be in the position 1...    
    # a_0, a_1, a_2, ..., a_n
    charpoly_array[n] = 1
    
    for r in range(1, n):
        # through n-1, then separate handling at the end for a_0
        a_idx = n - r
        running_sum = 0        
        
        working_matrix = working_matrix * A
        tracer_idx += 1
        traces_array[tracer_idx] = sp.trace(working_matrix)
        for j in range(1, r + 1):
            running_sum += charpoly_array[a_idx + j] * traces_array[j]
            # this itself can go in a memo I think... simplisitically with np.zeros and such... 
            # slightly more complicated with a buffer / pivot to which we don't go past
#         charpoly_array[a_idx] += - running_sum  * 1/ r
        charpoly_array[a_idx] += - sp.simplify(running_sum)  * Fraction(1, r)
    
    another_running_sum = 0
    for i in range(1, n + 1):
        another_running_sum += charpoly_array[i] * traces_array[i]
#     charpoly_array[0] = -1/n * another_running_sum 
    charpoly_array[0] = - Fraction(1,n) * sp.simplify(another_running_sum)
    determinant = charpoly_array[0]
    return charpoly_array.T, determinant

In [11]:
# B = sp.Matrix([[x, 2*x,  sin(3*x), 5*x],
#                [3*x, 4*x, sqrt(x), 6*x ],
#                [sqrt(x), sin(x), 3*sin(x), 7*x],
#                [sqrt(2*x), sin(3*x), 3*sin(4*x), 8*x]
#               ])
# B

B = sp.Matrix([[2*x, 2*y,  2*z, ],
               [4*x*y**2, 4*x**2*y, 0],
               [4*x**3 + 4*x*z**2, 4*y*z**2 + 4*y**3, 4*x**2*z + 4*z**3 +4*y**2*z]])
B

# an example from page 14 of 16 in MITRES_18_007_supp_notes07.pdf


Matrix([
[              2*x,               2*y,                          2*z],
[         4*x*y**2,          4*x**2*y,                            0],
[4*x**3 + 4*x*z**2, 4*y**3 + 4*y*z**2, 4*x**2*z + 4*y**2*z + 4*z**3]])

In [12]:
charpoly, determinant = get_char_pol_sym(B)
# print(charpoly)
print("\nand the determinant is\n")
sp.simplify(determinant)


and the determinant is



0

In [13]:
B.det()

0